In [2]:
import open3d as o3d
import numpy as np
import os
import pyvista as pv
import matplotlib.cm as cm
import matplotlib.pyplot as plt 
from sklearn.cluster import DBSCAN
import hdbscan

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [20]:
intput_file_path = r'region_growing_colored_B.pcd'
#intput_file_path = r'PointClouds\IntermediateFile_B\combined_segmented_cloud.pcd'
#intput_file_path = r'PointClouds\IntermediateFile_B\Roof.pcd'

In [6]:
# Load the PCD file
pcd = o3d.io.read_point_cloud(intput_file_path)
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)
# Check what's loaded
print(pcd)
print("Number of points:", len(pcd.points))
print("Has colors:", pcd.has_colors())


# Define a threshold for "red" (tune if needed)
red_threshold = 0.95  # close to 1.0 red channel
green_threshold = 0.2
blue_threshold = 0.2
# Create a mask for non-red points
non_red_mask = ~(
    (colors[:, 0] > red_threshold) &
    (colors[:, 1] < green_threshold) &
    (colors[:, 2] < blue_threshold)
)

# Filter points and colors
filtered_points = points[non_red_mask]
filtered_colors = colors[non_red_mask]
# Create a new point cloud without red points
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
filtered_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)

o3d.visualization.draw_geometries([filtered_pcd],
    window_name="Without Red Points",
    width=1024,
    height=768)

PointCloud with 117318 points.
Number of points: 117318
Has colors: True


# Visualized with Pyvista

In [ ]:
# Load the PCD file using Open3D
input_file_path = r'PointClouds\IntermediateFile\Roof_Clusters.pcd'
input_file_path = r'region_growing_colored.pcd'
pcd = o3d.io.read_point_cloud(input_file_path)

# Extract points and colors
points = np.asarray(pcd.points)
colors = (np.asarray(pcd.colors) * 255).astype(np.uint8)  # convert to 0–255


# -------- Step 2: Filter out red points --------
# Define thresholds for red
red_threshold = 0.95
green_threshold = 0.2
blue_threshold = 0.2

non_red_mask = ~(
    (colors[:, 0] > red_threshold) &
    (colors[:, 1] < green_threshold) &
    (colors[:, 2] < blue_threshold)
)

# Apply mask
filtered_points = points[non_red_mask]
filtered_colors = (colors[non_red_mask] * 255).astype(np.uint8)  # scale to 0–255

# Create a PyVista point cloud
cloud = pv.PolyData(filtered_points)
cloud['rgb'] = filtered_colors  # Assign RGB values

# Visualize in PyVista
plotter = pv.Plotter()
plotter.add_points(cloud, scalars='rgb', rgb=True, point_size=4, render_points_as_spheres=True)
plotter.add_axes()
plotter.show(window_size=[800, 800])

Widget(value='<iframe src="http://localhost:53290/index.html?ui=P_0x28ac3e58490_10&reconnect=auto" class="pyvi…

# Classify the Plane

In [8]:
pcd = o3d.io.read_point_cloud(intput_file_path)

points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# Remove red points
red_mask = ~((colors[:, 0] > 0.95) & (colors[:, 1] < 0.2) & (colors[:, 2] < 0.2))
filtered_points = points[red_mask]
filtered_colors = colors[red_mask]

## Calclulate the normal of the points in the same plane

In [35]:
def is_wall_by_quarter_normal_check(pcd_plane, threshold_deg=15, required_fraction=0.1):
    pcd_plane.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=5.0, max_nn=30)
    )

    normals = np.asarray(pcd_plane.normals)
    vertical = np.array([0, 0, 1])
    dot_products = np.abs(np.dot(normals, vertical))  # Cosine of angle with Z-axis
    angles = np.degrees(np.arccos(np.clip(dot_products, -1, 1)))

    vertical_like_count = np.sum((angles > (90 - threshold_deg)) & (angles < (90 + threshold_deg)))
    ratio = vertical_like_count / len(normals)

    return ratio >= required_fraction

In [36]:
# Unique colors after removing red
unique_colors = np.unique(filtered_colors, axis=0)

# Parameters
min_points_per_plane = 50
angle_thresh_deg = 10
normal_variance_thresh = 0.998

# Classification color map
class_colors = {
    'roof': [0, 1, 0],
    'wall': [1, 0, 0],
    'undulation': [0, 0, 1],
    'other': [1, 1, 0]
}

def classify_plane(normal, normal_variance, num_points):
    z_axis = np.array([0, 0, 1])
    angle = np.degrees(np.arccos(np.clip(np.dot(normal, z_axis), -1.0, 1.0)))
    if angle > 90 :
        angle = 180 - angle
    print(f"Normal: {normal}, Angle: {angle:.2f} degrees, Variance: {normal_variance:.4f}, Points: {num_points}")

    if num_points < min_points_per_plane:
        return 'undulation'
    elif angle < angle_thresh_deg:
        return 'roof'
    elif abs(angle - 90) < angle_thresh_deg:
        return 'wall'
    else:
        return 'roof'

segments = []
segments_points = []
plane_info = []
angles_per_plane = []
for i, color in enumerate(unique_colors):
    mask = np.all(filtered_colors == color, axis=1)
    pts = filtered_points[mask]

    if len(pts) < min_points_per_plane:
        continue

    plane = o3d.geometry.PointCloud()
    plane.points = o3d.utility.Vector3dVector(pts)
    plane.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=50.0, max_nn=10))
    
    normals = np.asarray(plane.normals)
    normal_mean = normals.mean(axis=0)
    normal_mean /= np.linalg.norm(normal_mean)

    plane.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=50.0, max_nn=5))
    normal_var = np.var(normals, axis=0).sum()

    plane_class = classify_plane(normal_mean, normal_var, len(pts))

    if plane_class == 'roof':
        if is_wall_by_quarter_normal_check(plane):
            plane_class = 'wall'
        else:
            plane_class = 'roof'

    plane.paint_uniform_color(class_colors[plane_class])
    segments.append(plane)
    plane_info.append((i, plane_class, len(pts), normal_mean, normal_var))

    # Store the normal mean and points for each segment
    angle = np.degrees(np.arccos(np.clip(np.dot(normal_mean, [0, 0, 1]), -1.0, 1.0)))
    if angle > 90:
        angle = 180 - angle  # Make symmetric around vertical
    angles_per_plane.append(angle)
    segments_points.append((pts, angle))

Normal: [0.24914709 0.26044086 0.93278952], Angle: 21.13 degrees, Variance: 0.9994, Points: 137
Normal: [ 0.00615254 -0.00624441  0.99996158], Angle: 0.50 degrees, Variance: 0.9669, Points: 12234
Normal: [ 0.02548343 -0.1146114   0.99308349], Angle: 6.74 degrees, Variance: 0.9651, Points: 461
Normal: [-0.21712962  0.00723364 -0.97611598], Angle: 12.55 degrees, Variance: 0.9895, Points: 544
Normal: [ 0.04564415 -0.03366147 -0.99839046], Angle: 3.25 degrees, Variance: 0.9961, Points: 2483
Normal: [ 0.03131844 -0.13217454 -0.99073157], Angle: 7.81 degrees, Variance: 0.7169, Points: 189
Normal: [ 0.97076897  0.23990975 -0.00713518], Angle: 89.59 degrees, Variance: 0.0985, Points: 284
Normal: [-0.32495152 -0.00625446  0.94570999], Angle: 18.97 degrees, Variance: 0.9708, Points: 104
Normal: [0.97045985 0.24115145 0.00732515], Angle: 89.58 degrees, Variance: 0.0324, Points: 104
Normal: [-0.08248521 -0.15218776 -0.98490359], Angle: 9.97 degrees, Variance: 0.9984, Points: 343
Normal: [0.1855028

# Visualized the Mean Normal of Plane

In [37]:
# Normalize angles for colormap
angles_array = np.array(angles_per_plane)
angle_min, angle_max = angles_array.min(), angles_array.max()
norm_angles = (angles_array - angle_min) / (angle_max - angle_min + 1e-8)

# Use a colormap (e.g., viridis)
cmap = cm.get_cmap("jet")

# ---- PyVista visualization ----
plotter = pv.Plotter()

for (pts, angle), norm_angle in zip(segments_points, norm_angles):
    rgb = cmap(norm_angle)[:3]  # Drop alpha
    cloud = pv.PolyData(pts)
    color_array = np.tile(rgb, (pts.shape[0], 1))
    cloud["colors"] = color_array
    plotter.add_points(cloud, scalars="colors", rgb=True, point_size=4, render_points_as_spheres=True)

plotter.add_axes()
plotter.show(title="Planes Colored by Mean Angle to Vertical", window_size=[1024, 768])

C:\Users\Vettel\AppData\Local\Temp\ipykernel_10280\1792319497.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("jet")


Widget(value='<iframe src="http://localhost:65429/index.html?ui=P_0x155d60534d0_12&reconnect=auto" class="pyvi…

# Visualized the Remove Wall Result

In [38]:
# Visualize the result
o3d.visualization.draw_geometries(segments, window_name="Plane Classification from Colored PCD")

# Print summary
print("\n--- Plane Classification Summary ---")
for idx, cls, count, normal, var in plane_info:
    print(f"Plane {idx}: {cls.upper():<12} | Points: {count:<5} | Normal: {normal.round(3)} | Var: {var:.4f}")


--- Plane Classification Summary ---
Plane 0: ROOF         | Points: 137   | Normal: [0.249 0.26  0.933] | Var: 0.9994
Plane 1: ROOF         | Points: 12234 | Normal: [ 0.006 -0.006  1.   ] | Var: 0.9669
Plane 2: ROOF         | Points: 461   | Normal: [ 0.025 -0.115  0.993] | Var: 0.9651
Plane 3: ROOF         | Points: 544   | Normal: [-0.217  0.007 -0.976] | Var: 0.9895
Plane 4: ROOF         | Points: 2483  | Normal: [ 0.046 -0.034 -0.998] | Var: 0.9961
Plane 5: ROOF         | Points: 189   | Normal: [ 0.031 -0.132 -0.991] | Var: 0.7169
Plane 6: WALL         | Points: 284   | Normal: [ 0.971  0.24  -0.007] | Var: 0.0985
Plane 7: ROOF         | Points: 104   | Normal: [-0.325 -0.006  0.946] | Var: 0.9708
Plane 8: WALL         | Points: 104   | Normal: [0.97  0.241 0.007] | Var: 0.0324
Plane 9: ROOF         | Points: 343   | Normal: [-0.082 -0.152 -0.985] | Var: 0.9984
Plane 10: ROOF         | Points: 788   | Normal: [0.186 0.07  0.98 ] | Var: 0.9643
Plane 11: WALL         | Points: 26

In [39]:
output_file_path = 'PointClouds\IntermediateFile_B\Roof.pcd'

# Combine all non-wall segments
non_wall_segments = [seg for seg, (_, cls, _, _, _) in zip(segments, plane_info) if cls != 'wall']

# Merge them into one PointCloud
merged = o3d.geometry.PointCloud()
for seg in non_wall_segments:
    merged += seg

# Save to a new PCD file
o3d.io.write_point_cloud(output_file_path, merged)
print(f"Saved non-wall planes to: {output_file_path}")

Saved non-wall planes to: PointClouds\IntermediateFile_B\Roof.pcd


# Visualized the Normal

In [9]:
# -------- Load PCD file --------
input_file_path = r'PointClouds\IntermediateFile\combined_segmented_cloud.pcd'
input_file_path = r'region_growing_colored_B.pcd'

pcd = o3d.io.read_point_cloud(input_file_path)

# Estimate normals
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.001, max_nn=10))

# Get numpy arrays
points = np.asarray(pcd.points)
normals = np.asarray(pcd.normals)

# Normalize normals
norms = np.linalg.norm(normals, axis=1, keepdims=True)
normals_normalized = normals / (norms + 1e-8)

# Calculate angle to vertical
z_axis = np.array([0, 0, 1])
dot_product = np.clip(normals_normalized @ z_axis, -1.0, 1.0)
angles_deg = np.degrees(np.arccos(dot_product))  # in degrees
angles_deg = 180- angles_deg  # Convert to angle from vertical

# Flip angles > 90° to reflect angle from vertical
angles_deg = np.where(angles_deg > 90, 180 - angles_deg, angles_deg)

# Normalize angle to [0,1] for colormap
normalized_angles = (angles_deg - 0) / (90 - 0 + 1e-8)

# Map to colors using matplotlib colormap
cmap = cm.get_cmap('jet')
colors = cmap(normalized_angles)[:, :3]  # Remove alpha

# -------- Convert to PyVista --------
cloud = pv.PolyData(points)
cloud['angle_deg'] = angles_deg
cloud['colors'] = (colors * 255).astype(np.uint8)

# Visualize
plotter = pv.Plotter()
plotter.add_points(cloud, scalars='angle_deg', cmap='jet', point_size=8, render_points_as_spheres=True)
#plotter.add_scalar_bar(title='Angle to Vertical (°)', n_labels=5)
plotter.show(window_size=[1024, 768])

C:\Users\Vettel\AppData\Local\Temp\ipykernel_31964\4148584097.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('jet')


Widget(value='<iframe src="http://localhost:53290/index.html?ui=P_0x28aaa12c710_3&reconnect=auto" class="pyvis…

# Classify the Roof

In [13]:
# Step 1: Load PCD
input_file_path = r'region_growing_colored.pcd'
pcd = o3d.io.read_point_cloud(input_file_path)
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# Optional Step 2: Remove red points
red_mask = (
    (colors[:, 0] > 0.95) &
    (colors[:, 1] < 0.2) &
    (colors[:, 2] < 0.2)
)
non_red_mask = ~red_mask
points = points[non_red_mask]

# Step 3: Project to XY plane
xy_points = points[:, :2]

# Step 4: Apply HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=500, min_samples=10, metric='euclidean', cluster_selection_epsilon=1)
labels = clusterer.fit_predict(xy_points)

n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Detected {n_clusters} clusters (excluding noise)")

# Step 5: Assign cluster colors
unique_labels = np.unique(labels)
cmap = plt.get_cmap("tab20", len(unique_labels))
colored_points = []
colored_rgb = []

for label in unique_labels:
    mask = labels == label
    cluster_pts = points[mask]
    
    # Gray for noise
    if label == -1:
        rgb = np.tile([0.5, 0.5, 0.5], (len(cluster_pts), 1))
    else:
        color = cmap(label % cmap.N)[:3]
        rgb = np.tile(color, (len(cluster_pts), 1))
    
    colored_points.append(cluster_pts)
    colored_rgb.append(rgb)

# Combine all clusters
all_points = np.vstack(colored_points)
all_colors = np.vstack(colored_rgb)

Detected 5 clusters (excluding noise)


In [14]:
# Step 6: Visualize with Open3D or PyVista (Open3D shown here)
pcd_out = o3d.geometry.PointCloud()
pcd_out.points = o3d.utility.Vector3dVector(all_points)
pcd_out.colors = o3d.utility.Vector3dVector(all_colors)

o3d.visualization.draw_geometries([pcd_out], window_name="HDBSCAN Result")

In [55]:
# Step 6: Create a new Open3D PointCloud and assign data
clustered_pcd = o3d.geometry.PointCloud()
clustered_pcd.points = o3d.utility.Vector3dVector(all_points)
clustered_pcd.colors = o3d.utility.Vector3dVector(all_colors)

# Step 7: Save the point cloud to a PCD file
output_path = r'PointClouds\IntermediateFile_B\Roof_clusters.pcd'
o3d.io.write_point_cloud(output_path, clustered_pcd)
print(f"Clustered point cloud saved to: {output_path}")

Clustered point cloud saved to: PointClouds\IntermediateFile_B\Roof_clusters.pcd
